# The smell of classes

## The "data bundle" smell

In [1]:
def momentum(mass, velocity):
    return mass * velocity

def energy(mass, velocity):
    return 0.5 * mass * velocity ** 2

def update_position(velocity, position, dt):
    return position + velocity * dt

In [2]:
# Naive
mass1 = 10.0
velocity1 = 0.9
position1 = 0.0

mass2 = 12.0
velocity2 = 0.1
position2 = -23.0

print(momentum(mass1, velocity1))
print(momentum(mass2, velocity2))
print(momentum(mass1, velocity2))  # ??

9.0
1.2000000000000002
1.0


We have three parameters that will be sent to these functions over and over again: `mass`, `velocity`, and `position`.

Moreover, the parameters cannot be mixed up (e.g. the velocity of one particle with the mass of another).

## Introducing classes as a data bundle template

In [61]:
class Particle:
    pass

particle1 = Particle(10.0, 0.9)
particle2 = Particle(12.0, 0.1, -23.0)

## Hands-on: Simple parameters class for a perceptron

Submit a PR for Issue #3 on GitHub.

1) Have a look at the "Hands-on Perceptron Step 0" notebook

2) Create a copy of the notebook, and get rid of the "parameters bundle" smell by writing a `Perceptron` class!

## Class methods

In [21]:
def momentum(particle):
    return particle.mass * particle.velocity

print(momentum(particle1))
print(momentum(particle2))

9.0
1.2000000000000002


In [30]:
class Particle:
    def __init__(self, mass, velocity, position=0.0):
        self.mass = mass
        self.velocity = velocity
        self.position = position
        
    # Method here

particle1 = Particle(10.0, 0.9, 0.0)
print(particle1.momentum())

9.0


We have been using class instances and methods all along...

In [54]:
s = 'A scanner Darkly'
s.capitalize()

'A scanner darkly'

In [92]:
x = set(['apple', 'banana', 'apple', 'pineapple'])
x

{'apple', 'banana', 'pineapple'}

In [91]:
x.union(['banana', 'kiwi'])

{'apple', 'banana', 'kiwi', 'pineapple'}

## Hands-on: The Perceptron class grows some methods

Submit a PR for Issue #3 on GitHub.

3) Have a look at the perceptron functions `update`, `activation`, and `predict` in the Hands-on Perceptron notebook, and rewrite them as methods of the class Perceptron

## Giving the class more responsibilities: serialization

Classes are a coeherent bundle of data and functions related to that data. Once a class is defined, with time it often acquires more responsibilities.

A common case in scientific code is the ability to save and load its data, aka "serialization".

In [1]:
import json


class Particle:
    def __init__(self, mass, velocity, position=0.0):
        self.mass = mass
        self.velocity = velocity
        self.position = position

    def momentum(self):
        return self.mass * self.velocity
    
    def energy(self):
        return 0.5 * self.mass * self.velocity ** 2
    
    # --- Serialization

    def to_json(self):
        """ Create a JSON record with all the particle data. """
        params = {
            'mass': self.mass,
            'velocity': self.velocity,
            'position': self.position,
        }
        return json.dumps(params)

In [2]:
particle = Particle(13.0, 0.9, position=3.2)
json_str = particle.to_json()

with open('model_v1.json', 'wt') as f:
    f.write(json_str)

In [3]:
!cat model_v1.json

{"velocity": 0.9, "position": 3.2, "mass": 13.0}

Now we need a corresponding method to re-create an instance from a JSON record.

A method that creates an instance is an alternative constructor, and is called a "factory method". We need to use a more advanced feature of classes: the `classmethod`. Class methods take the class instead of the `self` instance as their input, and return a fully formed instance.

In [7]:
class Particle:
    def __init__(self, mass, velocity, position=0.0):
        self.mass = mass
        self.velocity = velocity
        self.position = position

    def momentum(self):
        return self.mass * self.velocity
    
    def energy(self):
        return 0.5 * self.mass * self.velocity ** 2
    
    # --- Serialization

    def to_json(self):
        """ Create a JSON record with all the particle data. """
        params = {
            'mass': self.mass,
            'velocity': self.velocity,
            'position': self.position,
        }
        return json.dumps(params)
    
    @classmethod
    def from_json(cls, json_str):
        """ Create a Particle instance from a JSON record. """
        params = json.loads(json_str)
        return cls(**params)

In [8]:
with open('model_v1.json', 'rt') as f:
    json_str = f.read()
    
particle = Particle.from_json(json_str)

In [9]:
particle.mass

13.0

## Hands-on: Add serialization methods to the Perceptron

1) Add saving and loading methods to the Perceptron.

2) Save the parameters after training, and load them before the prediction part.

**Careful:**

1) JSON cannot handle NumPy arrays directly, you'll need to transform them in lists when you serialize, and back to arrays when you de-serialize.

2) You will probably notice that we have defined our constructor poorly so far... it will need to be rewritten!